# Topic Modeling and Visualization
We use [Latent Dirichlet Analysis (LDA)](https://dl.acm.org/doi/pdf/10.1145/2133806.2133826) to reveal thematic groups in the library data based on the "description" field of the library data.

We can then visualize the topics using [pyLDAvis](https://pypi.org/project/pyLDAvis/).

In [11]:
import nltk
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

pyLDAvis.enable_notebook()
stop_words = set(stopwords.words('english'))

In [12]:
catalog_df = pd.read_excel('data/workshop_data.xlsx')
catalog_df.sample(3)

,title,authors,year,OCLC,publisher,description,topic,sub_topic,sub_sub_topic,std_call_number,shelf,times_lent_since_2022,cover_file
5285,The behavior of sandwich structures of isotrop...,Jack R Vinson 1929-,1999,41220387,CRC Press,The Behavior of Sandwich Structures of Isotrop...,physics,applied mechanics,mechanics of civil structures general,EFF199,219,NaN,219/cover_41220387.webp
17168,Modern construction handbook.,Andrew Watts,2010,840444907,Springer,The Modern Construction Handbook has become a ...,building technology,building technology general,building technology general,UBA210,730,NaN,730/cover_840444907.webp
7013,Liquides aux interfaces = Liquids at interfaces,Ecole d'été de physique théorique (Les Houc...,1990,21762161,North Holland,This school was concerned with surface propert...,physics,physics of fluids,physics of liquids,FBB190,291,NaN,NaN


In [13]:
replacement_values = {"title": "",
                      "authors": "",
                      "OCLC": 0,
                      "publisher": "",
                      "description": "",
                      "topic": "",
                      "sub_topic": "",
                      "sub_sub_topic": "",
                      "std_call_number": "",
                      "shelf": 0,
                      "times_lent_since_2022": 0,
                      "cover_file": "",
                     }

catalog_df = catalog_df.fillna(value=replacement_values)

In [14]:
def remove_puncts(description_text, alphanumeric_only='True'):
    description_text = description_text.replace('-', ' ')
    clean_description_text = ''.join(e for e in description_text if e.isalnum() or e == ' ').lower()
    clean_description_text = ' '.join(clean_description_text.split())
    return clean_description_text


In [15]:
def get_words_tokenized_nopunct_nostop(descriptions, stop_w=stop_words):
    description_words_list = []
    for description in descriptions:
        clean_description = remove_puncts(description)
        words = word_tokenize(clean_description.lower())
        words_nostop = [word for word in words if not word in stop_w]
        description_words_list.append(words_nostop)
    return description_words_list

In [16]:
descriptions = catalog_df['description'].to_list()
tokenized_descriptions_list = get_words_tokenized_nopunct_nostop(descriptions)

In [17]:
dictionary = gensim.corpora.Dictionary(tokenized_descriptions_list)
corpus = [dictionary.doc2bow(description) for description in tokenized_descriptions_list]
topic_count = 25
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_count, id2word=dictionary, passes=50)

In [18]:
pyLDAvis.enable_notebook()
gensimvis.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.302450  0.026856       1        1  12.651873
12    -0.293932  0.127088       2        1  12.592210
14    -0.283242  0.048341       3        1  11.312464
9     -0.213885  0.045731       4        1   9.812366
24    -0.251612 -0.089928       5        1   8.476466
16    -0.224952 -0.028867       6        1   7.851366
3     -0.241063 -0.104761       7        1   7.618158
13    -0.182798  0.082040       8        1   5.502447
10    -0.170407  0.096745       9        1   5.276147
15    -0.046758 -0.287658      10        1   2.476603
18     0.041701 -0.210056      11        1   2.374154
8      0.062712 -0.119750      12        1   2.065521
5      0.057891  0.120206      13        1   2.030988
11     0.061508  0.147778      14        1   1.803088
2      0.087616 -0.017539      15        1   1.594291
22     0.159286  0.005549      16        1   1.346959
21     0.144591  0.013143      17        1   0.957879
1      0.146397  0.000936      18        1   0.888528
6      0.181644  0.017210      19        1   0.778974
20     0.180625 -0.017392      20        1   0.604388
7      0.209239  0.022165      21        1   0.505947
23     0.212482  0.027405      22        1   0.450007
17     0.218307  0.035337      23        1   0.372291
4      0.221390  0.029030      24        1   0.349564
19     0.225708  0.030390      25        1   0.307319, topic_info=            Term          Freq         Total Category  logprob  loglift
1417      design  10260.000000  10260.000000  Default  30.0000  30.0000
239          new   8335.000000   8335.000000  Default  29.0000  29.0000
75        theory   5533.000000   5533.000000  Default  28.0000  28.0000
6        edition   3894.000000   3894.000000  Default  27.0000  27.0000
4568   materials   3456.000000   3456.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
9473     seminar     20.293363     21.395648  Topic25  -5.2974   5.7321
5112      battle     19.932102     21.034387  Topic25  -5.3154   5.7312
2238    contrary     19.743310     20.845595  Topic25  -5.3249   5.7307
3962  generators     39.936904     49.480560  Topic25  -4.6204   5.5708
1363          ac     20.718657     35.489845  Topic25  -5.2766   5.2468

[1327 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
9387      14  0.978909      0
1299      14  0.998321      1
3801       5  0.125165     10
3801       7  0.094953     10
3801      14  0.776887     10
...      ...       ...    ...
3634      19  0.595933  youll
1435      16  0.996170   zijn
6728       7  0.986985   zone
9762      22  0.990860     zu
12482     25  0.963236    zum

[2446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 13, 15, 10, 25, 17, 4, 14, 11, 16, 19, 9, 6, 12, 3, 23, 22, 2, 7, 21, 8, 24, 18, 5, 20])